<a href="https://colab.research.google.com/github/mostafa-ja/sample/blob/master/PyTorch_Tutorial_10_Dataset_Transforms.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[link text](https://www.youtube.com/watch?v=X_QOZEko5uE&list=PLqnslRFeH2UrcDBWF5mfPGpqQDSta6VK4)




# __ call__ in Python

The __ call__ method enables Python programmers to write classes where the instances behave like functions and can be called like a function. 

**IMPORTANT :** When the instance is called as a function; if this method is defined, x(arg1, arg2, ...) is a shorthand for x.__call__(arg1, arg2, ...).

EX : model(x) ( which means model.call(x) )



In [1]:
class Product:
    def __init__(self):
        print("Instance Created")
  
    # Defining __call__ method
    def __call__(self, a, b):
        print(a * b)
  
# Instance created
ans = Product()
  
# __call__ method will be called
ans(10, 20)   # means ans.call(10,20)

Instance Created
200


**Difference between `__forward__` and `__call__` methods**

When you call something as class_object(params) it invokes the __ call__ method of that class.

If you dig the code of torch, specifically nn.Module you will see that __ call__ internally invokes forward but taking care of hooks and states that pytorch allows. 





In [2]:
import torch
import torchvision
from torch.utils.data import Dataset
import numpy as np

In [3]:
#data is (178, 14) and first column is y (1 or 2 or 3)
!wget https://raw.githubusercontent.com/python-engineer/pytorchTutorial/master/data/wine/wine.csv

--2022-08-11 08:59:02--  https://raw.githubusercontent.com/python-engineer/pytorchTutorial/master/data/wine/wine.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10889 (11K) [text/plain]
Saving to: ‘wine.csv’

wine.csv            100%[===================>]  10.63K  --.-KB/s    in 0s      

2022-08-11 08:59:02 (75.1 MB/s) - ‘wine.csv’ saved [10889/10889]



In [4]:
class WineDataset(Dataset):

  def __init__(self, transform=None):   # putting data for transform is optional when we write (None)

    xy = np.loadtxt('/content/wine.csv',delimiter=',',dtype=np.float32, skiprows=1)
    # note that we do not convert to tensor here
    self.x = xy[:,1:]
    self.y = xy[:,[0]]

    self.n_samples = xy.shape[0]

    self.transform = transform
    

  def __getitem__(self, index):
    sample = self.x[index], self.y[index]

    if self.transform :                 # if transform is not None , will be True
      sample = self.transform(sample)
    
    return sample


  def __len__(self):
    return self.n_samples
  

In [9]:
# we can make a custom transform or use [torchvision.transforms.ToTensor()]

class ToTensor:
  def __call__(self,sample):
    inputs, targets = sample
    return torch.from_numpy(inputs), torch.from_numpy(targets)


class MulTransform:
  def __init__(self,factor):
    self.factor = factor
  
  def __call__(self,sample):
    input,target = sample
    input *= self.factor
    return input,target


In [12]:
dataset = WineDataset() # OR dataset = WineDataset(transform=None)
input, label = dataset[0]
print(type(input),type(label))

<class 'numpy.ndarray'> <class 'numpy.ndarray'>


In [13]:
dataset = WineDataset(transform=ToTensor())
input, label = dataset[0]
print(type(input),type(label))
print(input,label)

<class 'torch.Tensor'> <class 'torch.Tensor'>
tensor([1.4230e+01, 1.7100e+00, 2.4300e+00, 1.5600e+01, 1.2700e+02, 2.8000e+00,
        3.0600e+00, 2.8000e-01, 2.2900e+00, 5.6400e+00, 1.0400e+00, 3.9200e+00,
        1.0650e+03]) tensor([1.])


In [11]:
# using two transforms together by torchvision.transforms.Compose()

composed = torchvision.transforms.Compose([ToTensor(),MulTransform(2)])
dataset = WineDataset(transform=composed)
input,label = dataset[0]
print(type(input),type(label))
print(input,label)


<class 'torch.Tensor'> <class 'torch.Tensor'>
tensor([2.8460e+01, 3.4200e+00, 4.8600e+00, 3.1200e+01, 2.5400e+02, 5.6000e+00,
        6.1200e+00, 5.6000e-01, 4.5800e+00, 1.1280e+01, 2.0800e+00, 7.8400e+00,
        2.1300e+03]) tensor([1.])
